In [1]:
# Modelling Libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
# NLP Libraries
import nltk
nltk.download('punkt')
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
# Utility Libraries
import pandas as pd
import random
import numpy as np
import json
import pickle
from pathlib import Path
import os
import utils
import time

[nltk_data] Downloading package punkt to /home/hanz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hanz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
ROOT = utils.get_project_root()
INPUT = ROOT / "input" 
CACHE = ROOT / "cache" # directory for saving trained models
OOS = INPUT / "oos-eval"
OOS_INPUT = OOS / "data"
print("ROOT: ", os.listdir(ROOT),"\n")
print("INPUT: ", os.listdir(INPUT),"\n")
print("CACHE: ", os.listdir(CACHE),"\n")
print("OOS: ", os.listdir(OOS),"\n")
print("OOS INPUT: ", os.listdir(OOS_INPUT),"\n")


ROOT:  ['notebooks', 'input', '.gitattributes', '.idea', 'README.md', 'venv', 'cache', '.git', '.ipynb_checkpoints', 'requirements.txt', '__init__.py', '.gitignore', 'docs', 'website', 'pychatbot'] 

INPUT:  ['dzone-intents.json', 'tim-intents.json', 'oos-eval'] 

CACHE:  ['chatbot_model.h5', 'oos-words.pkl', 'classes.pkl', 'words.pkl', 'oos-labels.pkl'] 

OOS:  ['paper.pdf', 'poster.pdf', 'README.md', 'hyperparameters.csv', 'LICENSE', '.gitignore', 'supplementary.pdf', 'clinc_logo.png', 'data'] 

OOS INPUT:  ['binary_undersample.json', 'binary_wiki_aug.json', 'data_full.json', 'data_small.json', 'all_wiki_sents.txt', 'data_imbalanced.json', 'data_oos_plus.json'] 



# Explatory Data Analysis: OOS

In [3]:
lemmatizer = WordNetLemmatizer()
# load data
with open( OOS_INPUT / "data_full.json", "r") as fh:
    data_dict = json.load(fh)
# df = pd.DataFrame(intents)
train_intents = data_dict['train']
test_intents = data_dict['test']
data_dict.keys()


dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])

In [4]:
train_intents[:2]

[['what expression would i use to say i love you if i were an italian',
  'translate'],
 ["can you tell me how to say 'i do not speak much spanish', in spanish",
  'translate']]

In [5]:
# len(train_intents)
df_train = pd.DataFrame(train_intents, columns = ["patterns", "intent"])
df_train.head()
# df_train.describe()

,patterns,intent
0,what expression would i use to say i love you ...,translate
1,can you tell me how to say 'i do not speak muc...,translate
2,"what is the equivalent of, 'life is good' in f...",translate
3,"tell me how to say, 'it is a beautiful morning...",translate
4,"if i were mongolian, how would i say that i am...",translate


In [6]:
df_train.shape

(15000, 2)

There are the usual patterns and intent; however, there is no response recorded. This indicates a lack of understanding as to how NLP training and test works. My initial impressions given previous experience with regression and classification tells me that there needs to be a target feature. What is the target feature(s)? The intent is a label of the pattern. Furthermore, the target feature is a translation of the predicted intent into an appropriate repsonse.<br>

The more I think about the problem that NLP solves, the more I can see why it's used in the sectors that need an automated call/response system. Although code is still needed to implement the functionality within the system, the rules that define the appropriate behavior are what changes and can be costly to refactor if they are hardcoded. Using properly labelled dataset with the appropriate action labels can be easier to maintain especially when there are large growing large inputs mapped to a particular intent of use.<br> 

It is most definitely appropriate to load up a sample dataset to understand the structure before diving into the full dataset. The reasoning is to reduce load times as well as to reduce the computational load when applying transformations. Furthermore, the full data set may adhere to the same format; however, that is not known until the full dataset is loaded in. Thus, some preparation such as reading descriptions about the datasets is well warranted to get some insight into the data.<br>
    

In [7]:

def tokenize(intents:list)->tuple:
    """
    """
    words, labels, documents = [], [], []
    # iterate list of tuples
    for i in intents:
        word = nltk.word_tokenize(i[0])
        intent = i[1]
        words.extend(word)
        documents.append((word, intent))
        if intent not in labels:
            labels.append(intent)
    return words, labels, documents

(words, labels, documents ) = tokenize(train_intents)

In [8]:
ignore_chars = "! ? , .".split(" ")

words = [lemmatizer.lemmatize(w.lower()) 
         for w in words if w not in ignore_chars]
words = sorted(list(set(words)))
labels = sorted(list(set(labels)))
# cache into binary files
with open(CACHE / "oos-words.pkl", "wb") as fh:
    pickle.dump(words, fh)
with open(CACHE/"oos-labels.pkl", "wb") as fh:
    pickle.dump(labels, fh)
print("Set Sizes")
print("Documents:",len(documents))
print("Labels:", len(labels))
print("Words:", len(words))

Set Sizes
Documents: 15000
Labels: 150
Words: 4831


In [9]:
x = []
Y_empty = [0]*len(labels) 

# considering the efficiency of this algo, is there a more efficient method? this
# is because as the data grows, computational time grows as well
DEBUG = False
start = time.time()
# iterate over each entry in documents
for d in documents:
    if DEBUG:
        print("Word pattern", d)
    bag = []
    word_patterns = d[0] # access the word pattern
    # lemmatize the entries in the selected word pattern
    lemma_word_patterns = [lemmatizer.lemmatize(w.lower()) for
                    w in word_patterns]
    if DEBUG:
        print("Lemmatized:", word_patterns)
        if (len(lemma_word_patterns) == len(word_patterns)):
            print("Identified case in which the collection of words were reduced to their core meaning")
    # convert the lemmatized word pattern to a numerical value
    for _ in words:
        bag.append(1) if _ in lemma_word_patterns else bag.append(0)
        
    # create the label for the labels in numeric form
    Y_row = list(Y_empty) 
    # set the activation on the output row
    Y_row[labels.index(d[1])] = 1
    if DEBUG:
        print("Numeric Class:",labels.index(d[1]))
        print("Y_row:",Y_row)
        print()
    x.append([bag, Y_row]) # add to the training set
end = time.time()
print("Completed dataset creation in {}s".format(end - start))
random.shuffle(x) # not sure if this is necessary? why shuffle the data when we create the training data with all of it and there is no test set
x = np.array(x)
x_train, Y_train = list(x[:,0]), list(x[:,1])


Completed dataset creation in 76.50162196159363s


<ipython-input-9-a9850971ac5a>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x = np.array(x)


In [10]:
start = time.time()
df_x_train  = pd.DataFrame(x_train)
end = time.time()
print("Completed dataset loading into pandas {}s".format(end - start))

print("Training input shape:",df_x_train.shape)
# x_train

Completed dataset loading into pandas 71.453298330307s
Training input shape: (15000, 4831)


In [11]:
start = time.time()
df_Y_train = pd.DataFrame(Y_train)
end = time.time()
print("Completed dataset loading into pandas {}s".format(end - start))

print("Training output shape",df_Y_train.shape)
# Y_train

Completed dataset loading into pandas 2.104707956314087s
Training output shape (15000, 150)


Consideration for the library used here for wrangling data is necessary. Potential alternatives are pyspark and koalas. Pyspark is faster but does not have the same interface naming conventions as pandas. Koalas applies a pandas interface for interacting with pyspark. 

# Model Creation

### Setup 

In [12]:
# use a deep neural network model
model = Sequential()
# add the 1st layer -- an input layer
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
# add the 2nd layer -- a hidden layer?
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# add the 3rd layer -- an output layer
model.add(Dense(len(Y_train[0]), activation='softmax'))
# compile the model. SGD + Nesterov accel gradient yields good results 
# for the particular model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

### Compilation & Freeze as Binary

In [13]:
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd,
              metrics=['accuracy'])
# train model
start = time.time()
hist = model.fit(np.array(x_train), 
                 np.array(Y_train),
                 epochs=200, 
                 batch_size=5,
                 verbose=1)
end = time.time()
print("Completed model fitting in {}s".format(end - start))
# cache model state into binary
model.save(CACHE/ "oos-chatbot_model.h5", hist)
print("Completed model caching...")

Epoch 1/200
3000/3000 [==============================] - 20s 6ms/step - loss: 4.7208 - accuracy: 0.0376
Epoch 2/200
3000/3000 [==============================] - 18s 6ms/step - loss: 2.7546 - accuracy: 0.2891
Epoch 3/200
3000/3000 [==============================] - 18s 6ms/step - loss: 2.2082 - accuracy: 0.4050
Epoch 4/200
3000/3000 [==============================] - 18s 6ms/step - loss: 1.9971 - accuracy: 0.4575
Epoch 5/200
3000/3000 [==============================] - 18s 6ms/step - loss: 1.9079 - accuracy: 0.4836
Epoch 6/200
3000/3000 [==============================] - 18s 6ms/step - loss: 1.8756 - accuracy: 0.4942
Epoch 7/200
3000/3000 [==============================] - 18s 6ms/step - loss: 1.8256 - accuracy: 0.5163
Epoch 8/200
3000/3000 [==============================] - 18s 6ms/step - loss: 1.8280 - accuracy: 0.5119
Epoch 9/200
3000/3000 [==============================] - 19s 6ms/step - loss: 1.7562 - accuracy: 0.5439
Epoch 10/200
3000/3000 [==============================] - 19s 6m

3000/3000 [==============================] - 8s 3ms/step - loss: 1.2234 - accuracy: 0.7153
Epoch 158/200
3000/3000 [==============================] - 8s 3ms/step - loss: 1.2692 - accuracy: 0.7124
Epoch 159/200
3000/3000 [==============================] - 11s 4ms/step - loss: 1.1551 - accuracy: 0.7245
Epoch 160/200
3000/3000 [==============================] - 8s 3ms/step - loss: 1.2233 - accuracy: 0.7172
Epoch 161/200
3000/3000 [==============================] - 8s 3ms/step - loss: 1.2215 - accuracy: 0.7150
Epoch 162/200
3000/3000 [==============================] - 8s 3ms/step - loss: 1.2022 - accuracy: 0.7205
Epoch 163/200
3000/3000 [==============================] - 10s 3ms/step - loss: 1.2065 - accuracy: 0.7169
Epoch 164/200
3000/3000 [==============================] - 8s 3ms/step - loss: 1.2071 - accuracy: 0.7218
Epoch 165/200
3000/3000 [==============================] - 8s 3ms/step - loss: 1.2379 - accuracy: 0.7180
Epoch 166/200
3000/3000 [==============================] - 11s 4ms/